 # 데이터는 Data.zip에서 다운받으세요.

In [1]:
cd C:\Users\USER\퀀트전략초보

C:\Users\USER\퀀트전략초보


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from dateutil.relativedelta import relativedelta 
warnings.filterwarnings(action = 'ignore')

In [3]:
path = '재무데이터.xlsx'
path_price = '수정주가.csv'
path_mkt = '시총데이터.csv'
path_endmonth = '결산월.csv'
path_kospiyn = '거래소데이터.csv'
path_delist_and_stop = '상장폐지데이터.csv'

In [4]:
def preprocessing_path(path) :
    ############################
    ## 재무 데이터 전처리     ##
    ############################
    df = pd.read_excel(path, header = [0,1], index_col = 0)
    return df.round(4)

def preprocessing_price(path_price) :
    ############################
    ## 가격 데이터 전처리     ##
    ############################
    x = pd.DataFrame([])
    for i in pd.read_csv(path_price, chunksize = 5000, parse_dates = ['Symbol'], index_col = 0) :
        x = pd.concat([x,i], axis = 0)
    return x.astype(np.float64)

def preprocessing_mktdata(path_mkt_data) :    
    ############################
    ## 시가총액 데이터 전처리 ##
    ############################
    mkt_data = pd.DataFrame([])
    for i in pd.read_csv(path_mkt_data , index_col = 0, chunksize = 12000, engine = 'python') :
        mkt_data = pd.concat([mkt_data, i], axis = 0)
    
    Common = mkt_data[mkt_data.columns[::2]].iloc[1:].applymap(lambda x : x.replace(',','') if type(x) == str else x).astype(np.float64)
    Prefer = mkt_data[mkt_data.columns[1::2]].iloc[1:].applymap(lambda x : x.replace(',','') if type(x) == str else x).astype(np.float64)
    
    mkt_data = pd.DataFrame(np.array(Common)/100 + np.array(Prefer.fillna(0))/100, columns = Common.columns, index = Common.index)
    mkt_data.index = pd.to_datetime(mkt_data.index)
    return mkt_data.resample('D').last().fillna(method = 'ffill', limit = 10)

def preprocessing_kospiyn(path_kospiyn, mkt='유가증권시장') :
    ##########################################
    ### 거래소(코스피,코스닥)데이터 전처리  ##
    #########################################
    data = pd.DataFrame([])
    for i in pd.read_csv(path_kospiyn, index_col = 0 , chunksize = 6000 , engine = 'python') :
        data = pd.concat([data,i], axis = 0)
    data.index = pd.to_datetime(data.index)
    if mkt == '유가증권시장' or mkt == '코스닥':
        data = data.astype(str).applymap(lambda x : int(mkt in x)).resample('D').last()
    elif mkt == 'both' :
        data = data.astype(str).applymap(lambda x : int('유가증권시장' in x or '코스닥' in x)).resample('D').last()
    data = data.fillna(method = 'ffill', limit = 30)
    return data

def preprocessing_stop_and_delist(path_delist_and_stop) :
    #########################################
    ## 거래정지 및 상장폐지 데이터 전처리  ##
    #########################################    
    data = pd.DataFrame([])
    for i in pd.read_csv(path_delist_and_stop, index_col = 0 , chunksize = 6000, engine = 'python') :
        data = pd.concat([data,i], axis = 0)
    delist_data = data[[data.columns[1]]].iloc[1:]
    delist_data = delist_data[delist_data['상장폐지일자'].isna() == False]
    delist_data['상장폐지일자'] = delist_data['상장폐지일자'].apply(lambda x : pd.to_datetime(x))
    small_col = list(data[data.columns[2:]].iloc[0])
    small_col = pd.to_datetime(small_col)
    big_col = pd.Series(data.columns[2:]).apply(lambda x : x.split('.')[0])    
    stop_data = data[data.columns[2:]].iloc[1:]
    stop_data.columns = [big_col,small_col]
    stop_data = stop_data.applymap(lambda x : 1 if x in ['TRUE' , True] else 0)        
    return  delist_data, stop_data

preprocessing_period = lambda path_endmonth : pd.read_csv(path_endmonth,index_col = 0, parse_dates=['Symbol']).fillna(method = 'ffill').fillna(method = 'bfill').fillna(12)

In [5]:
cleaned_data = preprocessing_path(path)
cleaned_price = preprocessing_price(path_price)
cleaned_kospiyn = preprocessing_kospiyn(path_kospiyn)
cleaned_mkt = preprocessing_mktdata(path_mkt)
delist_data, stop_data = preprocessing_stop_and_delist(path_delist_and_stop)
cleaned_endmonth = preprocessing_period(path_endmonth)

In [6]:
data_date = '2020-09-30'
today =  '2021-01-15'

 ## Value
 \begin{align}
PER = \frac{Mkt Value}{Earning} \\
PBR = \frac{Mkt Value}{Book} \\
PSR = \frac{Mkt Value}{Sales} \\
PCR = \frac{Mkt Value}{CFO} \\
EVEBITDA = \frac{EV}{EBITDA} \\
\end{align}

 ### 문제 1. 함수 Value는 PER, PBR, PSR, PCR, EVEBITDA의 순위와 평균순위를 호출해준다. 
 #### \ \ \ \ (PER>0 , PBR>0, PSR>0 , PCR>0 , EVEBITDA>0 조건이 성립하게 만들고 랭크를 나열하시오. 참고로 PER 등은 낮을수록 좋은 랭크를 받을 수 있도록 설정하시오.)
 
 #### 다음 표는 Value(cleaned_data, cleaned_mkt, data_date = '2020-09-30', today = '2021-01-15', n= 50) 을 입력했을 때 결과이다.
 #### 평균순위에 대하여 오름차순으로 표시되도록 함수를 만드세요.

| |PER|PBR|PSR|PCR|EBEBITDA|Total_Rank|
|------|---|---|---|---|---|---|
| A004960|15 |66 |23 |8 |83 |39|
| A001230|44 |38 |38 |33 |65 |43.6|
| A003960|19 |121 |13 |23 |44 |44|
| A006090|14 |NaN |65 |109 |3 |47.75|

In [7]:
def Value(cleaned_data, cleaned_mkt, data_date, today, n = 50) :
    ########################################
    ### 이곳에 코드를 작성하여 풀어주세요.##
    ########################################
    return 

 ### 문제 2. 2년보다 가격이 하락했고, 6개월 전보다는 가격이 상승하고, 1개월 전보다는 하락한 종목만 뽑으시오.
 
 momentum_screen(cleaned_price, today)

In [9]:
def momentum_screen(cleaned_price, today) :
    ########################################
    ### 이곳에 코드를 작성하여 풀어주세요.##
    ########################################
    return  

 ### Quality(1)
\begin{align}
Gross Profitability = \frac{Gross Profit}{Sales} \\
Operation Profitability = \frac{Operation Profit}{Sales} \\
ROE = \frac{Net Profit}{Equity} \\
ROA = \frac{Net Profit}{Asset} \\
\end{align}

 ### 문제 3. 함수 Quality는 매출총이익률, 영업이익률, ROE, ROA의 순위와 평균순위를 호출해준다. 
 #### (ROE, ROA, 영업이익률, 매출총이익은 높은것부터 낮은것 순으로 랭크가 매겨진다.)
 
 #### 다음 표는 Quality(cleaned_data, cleaned_mkt, data_date = '2020-09-30', today = '2021-01-15', n= 50) 을 입력했을 때 결과이다.
 #### 평균순위에 대하여 오름차순으로 표시되도록 함수를 만드세요.

| |GP|OP|ROE|ROA|Total_Rank|
|------|---|---|---|---|---|
| A123890|1 |11 |66 |59 |34.25 |
| A036570|29 |46 |51 |22 |37 |
| A192080|20 |47 |67 |20 |38.5 |
| A034310|47 |19 |72 |54 |48 |

In [11]:
def Quality(cleaned_data,cleaned_mkt, data_date, today , n = 50) :
    ########################################
    ### 이곳에 코드를 작성하여 풀어주세요.##
    ########################################
    return 

 ### 문제 4. Value 지표 5개와 Quality 지표 4개를 가지고 순위를 만들고 평균순위 순으로 Sort하시오.
 #### Value_Quality(cleaned_data, cleaned_mkt, data_date = '2020-09-30', today = '2021-01-15', n = 50) 

| |PER|PBR|PSR|PCR|EBEBITDA|GP|OP |ROE |ROA |Total_Rank |
|------|---|---|---|---|---|---|---|---|---|---|
| A123890|27 |203 |554 |3 |15 |1 |83 |22 |59 |104 |
| A272550|31 |nan |267 |83 |22 |254 |94 |41 |46 |104.75 |
| A006090|14 |nan |65 |109 |3 |389 |169 |56 |57 |107.75 |
| A013580|12 |151 |33 |13 |47 |441 |195 |27 |94 |112.56 |

In [13]:
def Value_Quality(cleaned_data,cleaned_mkt, data_date, today , n = 50) :
    ########################################
    ### 이곳에 코드를 작성하여 풀어주세요.##
    ########################################
    return 

 ### Growth 
\begin{align}
\Delta GP = GP_t - GP_{t-1y} \\
\Delta OP = OP_t - OP_{t-1y} \\
\Delta ROE = ROE_t - ROE_{t-1y} \\
\Delta ROA = ROA_t - ROA_{t-1y} \\
\Delta LEV = LEV_t - LEV_{t-1y} \\
\end{align}

### 문제 5. 문제 1,3과 같이 Total Rank를 만드시오. 부채비율은 Sort(ascending = True) 나머지는 False
Growth(cleaned_data, data_date, n = 50) 

| |dGP|dOP|dROE|dROA|dLEV|Total_Rank|
|------|---|---|---|---|---|---|
| A118000|26.5 |32.5 |12 |16 |10 |19.4|
| A064350|17 |27.5 |27.5 |17 |11 |23.2|
| A030790|62.5 |13.5 |13.5 |8 |13 |25.4|
| A008500|35.5 |3 |3 |10 |72 |33.3|

In [ ]:
def Growth(cleaned_data, data_date, n = 50) :
    ########################################
    ### 이곳에 코드를 작성하여 풀어주세요.##
    ########################################
    return 

 ### 문제 5.
Total_Rank = Value_Rank * 0.4 + Quality_Rank * 0.4 + Growth_Rank * 0.2
 
 Value_Quality_Growth(cleaned_data, data_date, today, n = 50) 


| |Value_Rank|Quality_Rank|Growth_Rank|Total_Rank|
|------|---|---|---|---|
| A006090|47.75	 |167.75 |262.9 |138.78 |
| A013580|51.2 |189.25 |220.4 |140.26 |
| A272550|100.75 |108.75 |307.5 |145.3 |
| A005320|186.4 |89.75 |176.9 |145.84 |

In [16]:
def Value_Quality_Growth(cleaned_data, cleaned_mkt, data_date, today, n = 50) :
    ########################################
    ### 이곳에 코드를 작성하여 풀어주세요.##
    ########################################
    return 

 ### 문제 6. Value_Quality_Growth 상위 N 개의 종목 중에 momentum_screen을 만족하는 종목을 추출하시오.
 
momentum_VQG(cleaned_data, cleaned_mkt, cleaned_price, data_date, today , n = 50)
 

In [18]:
def momentum_VQG(cleaned_data, cleaned_mkt, cleaned_price, data_date, today , n = 200 ) :
    ########################################
    ### 이곳에 코드를 작성하여 풀어주세요.##
    ########################################
    return 

### 문제 7.

 ### F Score (update)
 #### ----수익지표----
 #### if ROE > 전체 ROE 평균 : 1점
 #### if 영업이익/총자산 > (영업이익/총자산) 전체기업평균: 1점
 #### if CFOA > 0 : 1점
 #### if Accrual > 0 : 1점
 #### -----안정성지표-----
 #### dLEV < 0 : 1점
 #### d발행주식수 <= 0 : 1점
 #### d유동비율 > 0 : 1점
 #### -----효율성지표-----
 #### d매출총이익률 > 0 : 1점
 #### d(매출액/총자산) > 0 : 1점
 

In [1]:
def F_score(cleaned_data, data_date, today='', n = 50, cleaned_price = '', cleaned_mkt = '') :
    Data_Date = pd.to_datetime(data_date)
    if Data_Date >= pd.to_datetime('2001-12-31') :
        Data_bDate = pd.to_datetime(str(Data_Date.year -1) +'-'+ str(Data_Date.month)+'-'+str(Data_Date.day))
    else :
        if Data_Date.month < 5 :
            Data_bDate = pd.to_datetime(str(Data_Date.year -2) + '-12-31')
        else :
            Data_bDate = pd.to_datetime(str(Data_Date.year -1) +'-12-31')    
    
    
    #################
    ##나머지 완성 ###
    #################
    
    
    
    
    return F.sort_values(by = ['F_score'], ascending = False)